In [1]:
import numpy as np

In [2]:
import pickle

In [3]:
import Bbox

In [4]:
with open('representative_bbox_pickle_output/1uy5acKJCh.pickle','rb') as file:
    bboxes_list=pickle.load(file)

In [5]:
bboxes_list[2][0].frame_index

3

In [6]:
def findmax_score_frame(bbox_list,cls,alpha=1,beta=1):
    mixscore=0
    best_box=None
    whichindex=None
    whichframe=None
    
    for i, bboxes in enumerate(bbox_list):
        for j,bbox in enumerate(bboxes):
            if bbox.isnone:
                continue
            if bbox.cls==cls:
                candidate=alpha*bbox.motion*beta*bbox.score
                if candidate>mixscore:
                    mixscore=candidate
                    best_box=bbox
                    whichframe=i
                    whichindex=j
    return best_box,mixscore,whichindex,whichframe

In [7]:
def find_max_class_video(bboxes_list):
    n=np.zeros(2)
    for bboxes in bboxes_list:
        for bbox in bboxes:
            if bbox.isnone:
                continue
            n[bbox.cls]+=bbox.motion*bbox.score
    return np.argmax(n)


### add none bbox to all freams with bbox

In [8]:
def addnone(bboxes_list):
    for i,bboxes in enumerate(bboxes_list):
        judge=np.any([bbox.isnone for bbox in bboxes]) 
        if judge:
            pass
        else:
            bboxes.append(Bbox.bbox(None,None,None,None,i+1,isnone=True))
    return bboxes_list

In [9]:
bboxes_list=addnone(bboxes_list)


### find the max class for this video 

In [10]:
bestcls=find_max_class_video(bboxes_list)

In [11]:
bestbbox,score,boxes_index,frame_index=findmax_score_frame(bboxes_list,bestcls)

### filter by the best class


In [12]:
def filter_by_bestcls(bboxes_list,bestcls):
    for j,bboxes in enumerate(bboxes_list):
        for i,bbox in enumerate(bboxes):
            if bbox.isnone:
                continue
            else:
                if bbox.cls!=bestcls:
                    bboxes_list[j].pop(i)
    return bboxes_list

In [13]:
bboxes_list=filter_by_bestcls(bboxes_list,bestcls)

In [14]:
bboxes_list

[[<Bbox.bbox at 0x7f46c011ed68>],
 [<Bbox.bbox at 0x7f46c3578940>, <Bbox.bbox at 0x7f46c011edd8>],
 [<Bbox.bbox at 0x7f46c3578ba8>, <Bbox.bbox at 0x7f46d86ea4e0>],
 [<Bbox.bbox at 0x7f46c3578390>, <Bbox.bbox at 0x7f46d86ea240>],
 [<Bbox.bbox at 0x7f46c35784e0>, <Bbox.bbox at 0x7f46d86ea470>],
 [<Bbox.bbox at 0x7f46c35782e8>, <Bbox.bbox at 0x7f46d86eaa58>],
 [<Bbox.bbox at 0x7f46c35783c8>, <Bbox.bbox at 0x7f46d86ea630>],
 [<Bbox.bbox at 0x7f46c35781d0>, <Bbox.bbox at 0x7f46d86ea2e8>],
 [<Bbox.bbox at 0x7f46c3578dd8>, <Bbox.bbox at 0x7f46c3500748>],
 [<Bbox.bbox at 0x7f46d87e7cc0>, <Bbox.bbox at 0x7f46c35006a0>],
 [<Bbox.bbox at 0x7f46d87e74a8>, <Bbox.bbox at 0x7f46c3500208>],
 [<Bbox.bbox at 0x7f46d87e7f28>, <Bbox.bbox at 0x7f46c3500438>],
 [<Bbox.bbox at 0x7f46d87e7978>, <Bbox.bbox at 0x7f46c35786d8>],
 [<Bbox.bbox at 0x7f46d87e7f60>, <Bbox.bbox at 0x7f46c35785f8>],
 [<Bbox.bbox at 0x7f46d87e7b38>, <Bbox.bbox at 0x7f46c35784a8>],
 [<Bbox.bbox at 0x7f46d87e7518>, <Bbox.bbox at 0x7f46c35

In [24]:
a=np.arange(5)
if a[a>3]:
    print(4)

4


### interpotation 

In [15]:
def interp(bbox1,bbox2):
    interpnum=abs(bbox1.frame_index-bbox2.frame_index)
    def linefun(x,y,num):
        out=np.linspace(x,y,num=num,endpoint=False)
        out=out[1:]
        return out
    scores=linefun(bbox1.score,bbox2.score,interpnum)
    motions=linefun(bbox1.motion,bbox2.motion,interpnum)
    frames=linefun(bbox1.frame_index,bbox2.frame_index,interpnum)
    boxes=np.array((list(map(linefun,bbox1.box,bbox2.box,[interpnum]*4)))).transpose(1,0)
    bboxes=[]
    for i in range(interpnum-1):
        bboxes.append(Bbox.bbox(motions[i],scores[i],bbox1.cls,boxes[i],int(frames[i])))
    return bboxes        

def interpotation(bboxes_list):
    pointer=None
    counter=0
    allinterbox=[]
    for i in range(len(bboxes_list)):
        bboxes=bboxes_list[i]
        isnone=True if len(bboxes)==1 else False
        if isnone:
            counter+=1
            continue
        else:
            if counter==0:
                pointer=i
            else:
                if counter<10:
                    # could interp
                    if pointer is not None:
                        cad1=bboxes_list[pointer]
                        cad2=bboxes_list[i]
                        possible_score=[]
                        possible_pair=[]
                        for bbox1 in cad1:
                            if bbox1.isnone:
                                continue
                            for bbox2 in cad2:
                                if bbox2.isnone:
                                    continue
                                possible_score.append(bbox1.iouscore(bbox2))
                                possible_pair.append((bbox1,bbox2))
                        for i,score in enumerate(possible_score):
                            if score >-1:
                                bbox1=possible_pair[i][0]
                                bbox2=possible_pair[i][1]
                                bboxes=interp(bbox1,bbox2)
                                allinterbox.append(bboxes)               
            pointer=i
            counter=0
    return allinterbox
            
                
            
            
    
    

### add interpotation boxes to the bboxes_list

In [19]:
aggumentation=interpotation(bboxes_list)
for eaches in aggumentation:
    for each in eaches:
        i=each.frame_index-1
        bboxes_list[i].append(each)

In [17]:
def num(list):
    count=0
    for i in list:
        count+=len(i)
    return count

In [18]:
num(bboxes_list)

386

In [20]:
num(bboxes_list)

403

### split bboxes_list for forward and backwards

In [21]:
forward_bboxes_list=bboxes_list[frame_index:]
backward_bboxes_list=bboxes_list[:frame_index+1]
backward_bboxes_list=list(reversed(backward_bboxes_list))


In [22]:
def findway(process_list,bestbbox_score):
    max_score=[]
    max_score.append([bestbbox_score])
    max_score_index=[]
    max_score_index.append([0])
    
    for i,bboxes in enumerate(process_list[1:]):
        i=i+1
        score_list=[]
        index_list=[]
        if len(bboxes)==1:
            #current is none
            score_list.append(max(max_score[i-1]))
            if len(process_list[i-1])==1: # prev is also none
                index_list.append(0)
                
            else:
                index_list.append(np.argmax(max_score[i-1]))          
        else:
            if len(process_list[i-1])!=1 or i==1: #previous not None
                for bbox in bboxes:
                    if bbox.isnone:
                        emission=0
                    else:
                        emission=bbox.motion*bbox.score
                    transmissions=[]
                    # i==1 exception
                    
                    if i==1:
                        if bbox.isnone:
                            transmission=max_score[0][0]
                        else:
                            pre_bbox=process_list[0][0]
                            prev_bbox_index=0
                            transmission=bbox.iouscore(pre_bbox)+max_score[i-1][prev_bbox_index]
                        transmissions.append(transmission)
                    else:
                        for prev_bbox_index,pre_bbox in enumerate (process_list[i-1]):
                            if pre_bbox.isnone or bbox.isnone:
                                transmission=max_score[i-1]
                                transmission=transmission[prev_bbox_index]
                            else:
                                transmission=bbox.iouscore(pre_bbox)+max_score[i-1][prev_bbox_index]
                        transmissions.append(transmission)
                    score_list.append(np.max(transmissions)+emission) #could include a alpha or belta
                    index_list.append(np.argmax(transmissions))
            else:#previous is None
                for bbox in bboxes:
                    if bbox.isnone:
                        emission=0
                    else:
                        emission=bbox.motion*bbox.score
                    transmission=max_score[i-1][0]+0#iou is 0 for this case 
                    score_list.append(transmission+emission)
                    index_list.append(0)
        max_score.append(score_list)
        max_score_index.append(index_list)
    return max_score,max_score_index


In [23]:
backward_bboxes_list

[[<Bbox.bbox at 0x7f46d87e7b38>, <Bbox.bbox at 0x7f46c35784a8>],
 [<Bbox.bbox at 0x7f46c35788d0>, <Bbox.bbox at 0x7f46c00667b8>],
 [<Bbox.bbox at 0x7f46d87e7f60>, <Bbox.bbox at 0x7f46c35785f8>],
 [<Bbox.bbox at 0x7f46d87e7978>, <Bbox.bbox at 0x7f46c35786d8>],
 [<Bbox.bbox at 0x7f46c35003c8>, <Bbox.bbox at 0x7f46c0066860>],
 [<Bbox.bbox at 0x7f46d87e7f28>, <Bbox.bbox at 0x7f46c3500438>],
 [<Bbox.bbox at 0x7f46d87e74a8>, <Bbox.bbox at 0x7f46c3500208>],
 [<Bbox.bbox at 0x7f46d87e7cc0>, <Bbox.bbox at 0x7f46c35006a0>],
 [<Bbox.bbox at 0x7f46c3500390>, <Bbox.bbox at 0x7f46c0066588>],
 [<Bbox.bbox at 0x7f46c3578dd8>, <Bbox.bbox at 0x7f46c3500748>],
 [<Bbox.bbox at 0x7f46c35781d0>, <Bbox.bbox at 0x7f46d86ea2e8>],
 [<Bbox.bbox at 0x7f46d86ea6a0>, <Bbox.bbox at 0x7f46c0066668>],
 [<Bbox.bbox at 0x7f46c35783c8>, <Bbox.bbox at 0x7f46d86ea630>],
 [<Bbox.bbox at 0x7f46c35782e8>, <Bbox.bbox at 0x7f46d86eaa58>],
 [<Bbox.bbox at 0x7f46d86eaf98>, <Bbox.bbox at 0x7f46c00665c0>],
 [<Bbox.bbox at 0x7f46c35

In [24]:
max_score,max_score_index=findway(forward_bboxes_list,score)
max_score1,max_score_index1=findway(backward_bboxes_list,score)
print(len(max_score_index),len(max_score_index1))


336 24


In [25]:
def bestway(max_score,max_score_index):
    endpoint_index=np.argmax(max_score[-1])
    endpoint=max_score_index[-1][endpoint_index]
    best_path=[]
    best_path.append(endpoint)

    processed_max_index=list(reversed(max_score_index))[1:]
    for frame_index in range(len(processed_max_index)):

        endpoint=processed_max_index[frame_index][endpoint]
        best_path.append(endpoint)
    return list(reversed(best_path))

In [26]:
forwardpath=bestway(max_score,max_score_index)
backwardpath=bestway(max_score1,max_score_index1)
print(len(forwardpath),len(backwardpath))

336 24


### merger the forward and backward path  and find the trackets  

In [27]:
backwardpath=list(reversed(backwardpath))
backwardpath.pop()
forwardpath.pop(0)
print(len(forwardpath),len(backwardpath))
allpath=backwardpath+[boxes_index]+forwardpath

335 23


In [28]:
len(bboxes_list)

359

### check not none number

In [29]:
def check_not_none(bboxes_list):
    allcount=0
    for bboxes in bboxes_list:
        assert len(bboxes)>=1
        allcount+=len(bboxes)-1
    return allcount

In [44]:
check_not_none(bboxes_list)

27

### update bbox_list by finded path

In [31]:
for i in range(len(allpath)):
    bboxes_list[i].pop(allpath[i])


In [32]:
#add none
bboxes_list=addnone(bboxes_list)

In [33]:
check_not_none(bboxes_list)

18

In [43]:
num(bboxes_list)

386

In [45]:
aggumentation=interpotation(bboxes_list)
for eaches in aggumentation:
    for each in eaches:
        i=each.frame_index-1
        bboxes_list[i].append(each)